In [ ]:
!pip install mysql-connector-python




In [ ]:
import mysql.connector

mycheckdb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",

)

print(mycheckdb)
mycursor = mycheckdb.cursor(buffered=True)

load the data 

In [ ]:
import pandas as pd
df=pd.read_csv("traffic_stops - traffic_stops_with_vehicle_number.csv")

In [ ]:
display(df)

Data preprocessing 

In [ ]:
#Drop columns with all missing values
df.dropna(axis=1, how='all', inplace=True)

# Fill missing values (one line)
df.fillna({
    'driver_age': df['driver_age'].median(),
    'driver_gender': 'unknown',
    'driver_race': 'unknown',
    'violation': 'unknown',
    'stop_outcome': 'unknown',
    'search_type': 'unknown'
}, inplace=True)

# Combine date and time columns into a single datetime (timestamp)
df['timestamp'] = pd.to_datetime(df['stop_date'] + ' ' + df['stop_time'])

In [ ]:

mycursor.execute("create database Police_Check_new")

In [ ]:
mycursor.execute("USE Police_Check_new")

In [ ]:
!pip install tabulate

In [ ]:
mycursor.execute("""
CREATE TABLE IF NOT EXISTS Police_Check_new.cleaned_traffic_stops (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    is_arrested BOOLEAN,
    drugs_related_stop BOOLEAN,
    stop_duration VARCHAR(50),
    vehicle_number VARCHAR(50),
    timestamp DATETIME               -- 🕒 Combined date and time
)
""")

In [ ]:

#Drop columns with all missing values
df.dropna(axis=1, how='all', inplace=True)
# Fill missing values 
df.fillna({
    'driver_age': df['driver_age'].median(),
    'driver_gender': 'unknown',
    'driver_race': 'unknown',
    'violation': 'unknown',
    'stop_outcome': 'unknown',
    'search_type': 'unknown'
}, inplace=True)

# Combine date and time columns into a single datetime (timestamp)
df['timestamp'] = pd.to_datetime(df['stop_date'] + ' ' + df['stop_time'])

In [ ]:
df

In [ ]:
df['stop_date'] = pd.to_datetime(df['stop_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df['stop_time'] = pd.to_datetime(df['stop_time'], errors='coerce').dt.strftime('%H:%M:%S')

df=df.where(pd.notnull(df),None)

insert_query = """
INSERT INTO Police_Check_new.cleaned_traffic_stops (
    stop_date, stop_time, country_name, driver_gender,
    driver_age, driver_race, violation,
    search_conducted, search_type, stop_outcome,
    is_arrested, stop_duration, drugs_related_stop,
    vehicle_number, timestamp
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for _, row in df.iterrows():
    mycursor.execute(insert_query, (
        row.get('stop_date'),
        row.get('stop_time'),
        row.get('country_name'),
        row.get('driver_gender'),
        int(row.get('driver_age')) if row.get('driver_age') not in [None, ''] else None,
        row.get('driver_race'),
        row.get('violation'),
        bool(row.get('search_conducted')) if row.get('search_conducted') not in [None, ''] else None,
        row.get('search_type'),
        row.get('stop_outcome'),
        bool(row.get('is_arrested')) if row.get('is_arrested') not in [None, ''] else None,
        row.get('stop_duration'),
        bool(row.get('drugs_related_stop')) if row.get('drugs_related_stop') not in [None, ''] else None,
        row.get('vehicle_number'),
        row.get('timestamp')
    ))

mycheckdb.commit()
print("✅ All records inserted successfully!")

In [ ]:
import pandas as pd
import mysql.connector
mycursor.execute("select * from Police_Check_new.cleaned_traffic_stops")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
mycursor.execute("SELECT * FROM Police_Check_new.cleaned_traffic_stops")
out = mycursor.fetchall()

# Convert to DataFrame
df = pd.DataFrame(out, columns=[i[0] for i in mycursor.description])

# Save as CSV file
df.to_csv("cleaned_traffic_stops.csv", index=False)

print("CSV file saved successfully!")

new_police_c['stop_date'] = pd.to_datetime(new_police_c['stop_date'], format='%Y-%m-%d', errors='coerce').dt.strftime('%d-%m-%Y')
new_police_c['stop_time'] = pd.to_datetime(new_police_c['stop_time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

sql = """
INSERT INTO Police_Check_new.cleaned_traffic_stops (
    stop_date, stop_time, country_name, driver_gender, driver_age_raw,
    driver_age, driver_race, violation_raw, violation, search_conducted,
    search_type, stop_outcome, is_arrested, stop_duration,
    drugs_related_stop, vehicle_number, timestamp
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for _, row in new_police_c.iterrows():
    mycursor.execute(sql, (
        row['stop_date'],
        row['stop_time'],
        row['country_name'],
        row['driver_gender'],
        row['driver_age_raw'],
        row['driver_age'],
        row['driver_race'],
        row['violation_raw'],
        row['violation'],
        row['search_conducted'],
        row['search_type'],
        row['stop_outcome'],
        row['is_arrested'],
        row['stop_duration'],
        row['drugs_related_stop'],
        row['vehicle_number'],
        row['timestamp']
    ))
    